SMS Spam Classification

Natural Language Processing Using Machine Learning

In [43]:
#Loading the libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [20]:
#Reading the csv file
messages = pd.read_csv('https://raw.githubusercontent.com/Shubh-33/My_Projects_beginner_ML/main/sms%20classifier/spam.csv',encoding = 'latin')

#Specifying the names of the columns while reading csv file (tsv--tab separated values)

In [ ]:
messages

### Exploratory Data Analysis

In [22]:
#Info about the data
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [23]:
#Finding missing values
messages.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [24]:
#Target variables counts
messages['v1'].value_counts()

#Data is imbalanced but for now we will continue with this

ham     4825
spam     747
Name: v1, dtype: int64

## Data Preprocessing

 renaming the columns

In [25]:
messages = messages.rename({'v1':'label', 'v2':'message'},axis='columns')
messages

,label,message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [26]:
data = messages.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis = 1)
data

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [27]:
#Calculating length of message
mes_len=0
length=[]
for i in range(len(data)):
    length.append(len(data['message'][i]))

In [ ]:
length

In [29]:
#Adding Length column to the dataframe
data['Length']=length

In [30]:
data.head()

,label,message,Length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [31]:
import string
count=0
punct=[]
for i in range(len(data)):
    for j in messages['message'][i]:
        if j in string.punctuation:
            count+=1
    #print(count)
    punct.append(count)
    count=0

In [ ]:
punct

In [35]:
#Adding punctuation length column to dataframe
data["Punctuation"]=punct

## Text Cleaning

In [36]:
#Regex
import re

#Stopwords
from nltk.corpus import stopwords

#Lemmatization
from nltk.stem import WordNetLemmatizer
#Creating object for Lemmatizer
lemmatizer = WordNetLemmatizer()

In [45]:
#Removal of extra characters and stop words and lemmatization
corpus = []

#Skipping the 0th index (it's of Label)
for i in range(0,len(data)):
    words = re.sub('[^a-zA-Z]',' ',data['message'][i])
    words = words.lower()
    #Splits into list of words
    words = words.split()

    #Lemmatizing the word and removing the stopwords
    words = [lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]

    #Again join words to form sentences
    words = ' '.join(words)

    corpus.append(words)

In [46]:
corpus[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [47]:
#Replacing Original Message with the Transformed Messages
data['message'] = corpus

In [48]:
data.head()

,label,message,Length,Punctuation
0,ham,go jurong point crazy available bugis n great ...,111,9
1,ham,ok lar joking wif u oni,29,6
2,spam,free entry wkly comp win fa cup final tkts st ...,155,6
3,ham,u dun say early hor u c already say,49,6
4,ham,nah think go usf life around though,61,2


# Analyzing the difference between Spam and Ham messages

In [51]:
spam_messages =data[data['label'] == 'spam']
ham_messages = data[data['label'] == 'ham']

In [52]:
spam_messages.head()

,label,message,Length,Punctuation
2,spam,free entry wkly comp win fa cup final tkts st ...,155,6
5,spam,freemsg hey darling week word back like fun st...,148,8
8,spam,winner valued network customer selected receiv...,158,6
9,spam,mobile month u r entitled update latest colour...,154,2
11,spam,six chance win cash pound txt csh send cost p ...,136,8


In [53]:
ham_messages.head()

,label,message,Length,Punctuation
0,ham,go jurong point crazy available bugis n great ...,111,9
1,ham,ok lar joking wif u oni,29,6
3,ham,u dun say early hor u c already say,49,6
4,ham,nah think go usf life around though,61,2
6,ham,even brother like speak treat like aid patent,77,2


In [54]:
spam_messages['Length'].mean()

138.8661311914324

In [55]:
ham_messages['Length'].mean()

71.02362694300518

We can see that Spam messages have more average words than Ham messages

# Model Building

In [56]:
X = data['message']
y = data['label']

#### Train Test Split

In [57]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , y, test_size = 0.33, random_state = 42)
X_train.head()

3235    aight text back mu swing need somebody get door
945                          cant wait see photo useful
5319                            kothi print marandratha
5528                           effect irritation ignore
247                           kallis wont bat nd inning
Name: message, dtype: object

### Demonstration of Count Vectorizer
(Bag of Words)

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()
X_train_count_vect=count_vect.fit_transform(X_train).toarray()
X_train_count_vect

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [59]:
# 3733 are the sentences and 5772 are the words in total sentences
X_train_count_vect.shape

(3733, 5728)

#### Note:-
There might be that, some words in 5772 words are not frequently present and are just appearing 1-2 times, we can reduce them using cv = CountVectorizer(max_features = 4000) (an approach)

This will only take 4000 words leading to coming of most frequent words

We can change the max_features, according to what we want

## Demonstration of TF-IDF Vectorizer
(Term Frequency - Inverse Document Frequency)

CountVectorizer(Bag of Words) + TFIDF Transformer, Scikit-Learn has provided with a method of TFIDF vectorizer (combining two steps into one)

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf_vect=count_vect.fit_transform(X_train).toarray()
X_train_tfidf_vect

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [61]:
X_train_tfidf_vect.shape

(3733, 5728)

## Pipelining

We are doing pipelining as we need to perform the same procedures for the test data to get predictions, that may be tiresome.

However what convenient about this pipeline object is that it actually can perform all these steps for you in a single cell, that means you can directly provide the data and it will be both vectorized and run the classifier on it in a single step.

Pipeline takes list of tuple.

In [62]:
from sklearn.pipeline import Pipeline

## Naive Bayer Classifier

In [63]:
from sklearn.naive_bayes import MultinomialNB
#each tuple takes the name you decide , next you call what you want to occur
text_mnb=Pipeline([('tfidf',TfidfVectorizer()),('mnb',MultinomialNB())])
#Now u can directly pass the X_train dataset.
text_mnb.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('mnb', MultinomialNB())])

In [64]:
#It will take the X_test and do all the steps, vectorize it and predict it
y_preds_mnb=text_mnb.predict(X_test)
#Predictions of the test data
y_preds_mnb

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype='<U4')

In [65]:
#Training score
text_mnb.score(X_train,y_train)

0.9788373961960889

In [66]:
#Testing score
text_mnb.score(X_test,y_test)

0.9635671560630777

Evaluation Metrics

In [67]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_preds_mnb))

[[1586    1]
 [  66  186]]


In [68]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_preds_mnb))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1587
        spam       0.99      0.74      0.85       252

    accuracy                           0.96      1839
   macro avg       0.98      0.87      0.91      1839
weighted avg       0.96      0.96      0.96      1839



## SVM Classifier

In [69]:
from sklearn.svm import LinearSVC
#each tuple takes the name you decide , next you call what you want to occur
text_svm=Pipeline([('tfidf',TfidfVectorizer()),('svm',LinearSVC())])
#Now u can directly pass the X_train dataset.
text_svm.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('svm', LinearSVC())])

In [70]:
#It will take the X_test and do all the steps, vectorize it and predict it
y_preds_svm=text_svm.predict(X_test)
#Predictions of the test data
y_preds_svm

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'spam'], dtype=object)

In [71]:
#Training score
text_svm.score(X_train,y_train)

1.0

In [72]:
#Testing score
text_svm.score(X_test,y_test)

0.9815116911364872

#### Evaluation Metrics

In [73]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_preds_svm))

[[1581    6]
 [  28  224]]


In [74]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_preds_svm))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1587
        spam       0.97      0.89      0.93       252

    accuracy                           0.98      1839
   macro avg       0.98      0.94      0.96      1839
weighted avg       0.98      0.98      0.98      1839



## Prediciting on New SMS

In [75]:
text = 'Congratulations, you have won a lottery of $5000. To Won Text on,555500 '
def refined_text(text):
    #Removal of extra characters and stop words
    words = re.sub('[^a-zA-Z]',' ',text)
    words = words.lower()
    #Splits into list of words
    words = words.split()

    #Lemmatizing the word and removing the stopwords
    words = [lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]

    #Again join words to form sentences
    words = ' '.join(words)
    return words

In [76]:
refined_word = refined_text(text)
refined_word = [refined_word]
refined_word

['congratulation lottery text']

In [77]:
# Directly predicting the single message to the model
text_mnb.predict(refined_word)

array(['spam'], dtype='<U4')